In [3]:
# parallel.py
from __future__ import division
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.opt.parallel import SolverManagerFactory
import sys

action_handle_map = {} # maps action handles to instances

# Create a solver
optsolver = SolverFactory('ipopt')

# create a solver manager
# 'pyro' could be replaced with 'serial'
solver_manager = SolverManagerFactory('pyro')
if solver_manager is None:
    print("Failed to create solver manager.")
    sys.exit(1)

#
# A simple model with binary variables and
# an empty constraint list.
#
model = AbstractModel()
model.n = Param(default=4)
model.x = Var(RangeSet(model.n), within=Binary)
def o_rule(model):
    return summation(model.x)
model.o = Objective(rule=o_rule)
model.c = ConstraintList()

# Create two model instances
instance1 = model.create()

instance2 = model.create()
instance2.x[1] = 1
instance2.x[1].fixed = True

# send them to the solver(s)
action_handle = solver_manager.queue(instance1, opt=optsolver, warmstart=False, tee=True, verbose=False)
action_handle_map[action_handle] = "Original"
action_handle = solver_manager.queue(instance2, opt=optsolver, warmstart=False, tee=True, verbose=False)
action_handle_map[action_handle] = "One Var Fixed"

# retrieve the solutions
for i in range(2): # we know there are two instances
    this_action_handle = solver_manager.wait_any()
    solved_name = action_handle_map[this_action_handle]
    results = solver_manager.get_results(this_action_handle)
    print( "Results for",solved_name)
    print(results)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Results for",solved_name)? (<ipython-input-3-8822713492a9>, line 50)

In [25]:
import json
import os

path = os.path.join(os.getcwd(), "utils", "32da813acb9e", "Input.registry.file")
with open(path, "r") as file:
    input_config_file = json.loads(file.read())

input_config_mqtt = {}
model_name = "CarParkModel"

defined_external_names = ['SoC_Value']
defined_non_prediction_names =  ['P_PV']
defined_prediction_names =  []
external_names =  []

prediction_names = []
non_prediction_names = []
external_names = []
generic_names = []

def add_name_to_list(key):
    if key in defined_prediction_names:
        prediction_names.append(key)
    elif key in defined_non_prediction_names:
        non_prediction_names.append(key)
    elif key in defined_external_names:
        external_names.append(key)
    else:
        generic_names.append(key)

model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}

class Constants:

    ESS = "ESS"
    SoC_Value = "SoC_Value"
    meta = "meta"
    P_PV = "P_PV"
    P_Load = "P_Load"
    Q_Load = "Q_Load"
    mqtt = "mqtt"
    Cars = "Cars"
    Charging_Station = "Charging_Station"


def extract_optimization_values():
    data = {}
    for input_config in [input_config_file, input_config_mqtt]:
        for k, v in input_config.items():
            if isinstance(v, dict):
                for k1, v1 in v.items():
                    if k1 == Constants.meta:
                        for k2, v2 in v1.items():
                            try:
                                v2 = float(v2)
                            except ValueError:
                                pass
                            if isinstance(v2, float) and v2.is_integer():
                                v2 = int(v2)
                            if k2 in model_variables.keys():
                                indexing = model_variables[k2]["indexing"]
                                if indexing == "index":
                                    data[k2] = {int(0): v2}
                                elif indexing == "None":
                                    data[k2] = {None: v2}
                            else:
                                data[k2] = {None: v2}
                    if k1 == Constants.Cars:
                        for car in v1:
                            print(car)
                    if k1 == Constants.Charging_Station:
                        print(k1)
                    elif k1 == Constants.SoC_Value and isinstance(v1, int):
                        indexing = model_variables[Constants.SoC_Value]["indexing"]
                        if indexing == "index":
                            data[Constants.SoC_Value] = {int(0): float(v1 / 100)}
                        elif indexing == "None":
                            data[Constants.SoC_Value] = {None: float(v1 / 100)}
                    elif isinstance(v1, list):
                        add_name_to_list(k1)
                    elif k == "generic" and not isinstance(v1, dict):
                        try:
                            v1 = float(v1)
                        except ValueError:
                            pass
                        if isinstance(v1, float) and v1.is_integer():
                            v1 = int(v1)
                        data[k1] = {None: v1}
    return data

data = extract_optimization_values()
data

In [26]:
path = os.path.join(os.getcwd(), "utils", "32da813acb9e", "Input.registry.file")
with open(path, "r") as file:
    input_config_file = json.loads(file.read())

input_config_mqtt = {}
model_name = "CarParkModel"

In [27]:
defined_external_names = ['SoC_Value']
defined_non_prediction_names =  ['P_PV']
defined_prediction_names =  []
external_names =  []

prediction_names = []
non_prediction_names = []
external_names = []
generic_names = []

def add_name_to_list(key):
    if key in defined_prediction_names:
        prediction_names.append(key)
    elif key in defined_non_prediction_names:
        non_prediction_names.append(key)
    elif key in defined_external_names:
        external_names.append(key)
    else:
        generic_names.append(key)

In [44]:
model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}

class Constants:

    ESS = "ESS"
    SoC_Value = "SoC_Value"
    meta = "meta"
    P_PV = "P_PV"
    P_Load = "P_Load"
    Q_Load = "Q_Load"
    mqtt = "mqtt"
    Cars = "Cars"
    Charging_Station = "Charging_Station"


In [54]:
def extract_optimization_values():
    data = {}
    for input_config in [input_config_file, input_config_mqtt]:
        for k, v in input_config.items():
            if isinstance(v, dict):
                for k1, v1 in v.items():
                    if k1 == Constants.meta:
                        for k2, v2 in v1.items():
                            try:
                                v2 = float(v2)
                            except ValueError:
                                pass
                            if isinstance(v2, float) and v2.is_integer():
                                v2 = int(v2)
                            if k2 in model_variables.keys():
                                indexing = model_variables[k2]["indexing"]
                                if indexing == "index":
                                    data[k2] = {int(0): v2}
                                elif indexing == "None":
                                    data[k2] = {None: v2}
                            else:
                                data[k2] = {None: v2}
                    if k1 == Constants.Cars:
                        for car in v1:
                            print(car)
                    if k1 == Constants.Charging_Station:
                        print(k1)
                    elif k1 == Constants.SoC_Value and isinstance(v1, int):
                        indexing = model_variables[Constants.SoC_Value]["indexing"]
                        if indexing == "index":
                            data[Constants.SoC_Value] = {int(0): float(v1 / 100)}
                        elif indexing == "None":
                            data[Constants.SoC_Value] = {None: float(v1 / 100)}
                    elif isinstance(v1, list):
                        add_name_to_list(k1)
                    elif k == "generic" and not isinstance(v1, dict):
                        try:
                            v1 = float(v1)
                        except ValueError:
                            pass
                        if isinstance(v1, float) and v1.is_integer():
                            v1 = int(v1)
                        data[k1] = {None: v1}
    return data

In [55]:
data = extract_optimization_values()
data

{'Car_Name': 'Car1', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car2', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car3', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car4', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car5', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car6', 'Battery_Capacity_kWh': 30}
{'Car_Name': 'Car7', 'Battery_Capacity_kWh': 30}
Charging_Station


{'ESS_Capacity': {None: 2430},
 'ESS_Max_Charge_Power': {None: 0.62},
 'ESS_Max_Discharge_Power': {None: 0.62},
 'ESS_Max_SoC': {None: 1},
 'ESS_Min_SoC': {None: 0.2},
 'Behavior_Model': {None: 'null'},
 'Initial_ESS_SoC': {None: 'null'},
 'Initial_VAC_SoC': {None: 'null'},
 'Number_of_Parked_Cars': {None: 7},
 'Unit_Consumption_Assumption': {None: 2.5},
 'Unit_Drop_Penalty': {None: 1},
 'VAC_Capacity': {None: 'null'},
 'Value': {None: 'null'},
 'Max_Voltage_Drop': {None: 1.2315135},
 'Min_Voltage_Drop': {None: 1.0246457},
 'P_Grid_Max_Export_Power': {None: 10},
 'Q_Grid_Max_Export_Power': {None: 10},
 'pf_Load': {None: 1},
 'PV_Inv_Max_Power': {None: 3501}}

In [6]:
x = None

assert x, "Error"

AssertionError: Error